In [10]:
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import os
import sqlite3
from sqlalchemy import create_engine
import time

In [12]:
usd_files = [
"AVTUSD.parquet",
"BCHUSD.parquet",
"BTCUSD.parquet",
"DSHUSD.parquet",
"EDOUSD.parquet",
"EOSUSD.parquet",
"ETCUSD.parquet",
"ETHUSD.parquet",
"ETPUSD.parquet",
"IOTUSD.parquet",
"LTCUSD.parquet",
"NEOUSD.parquet",
"OMGUSD.parquet",
"QTMUSD.parquet",
"RRTUSD.parquet",
"SANUSD.parquet",
"XMRUSD.parquet",
"XRPUSD.parquet",
"ZECUSD.parquet"]

In [13]:
btc_files = [
"AGIBTC.parquet",
"AIDBTC.parquet",
"AIOBTC.parquet",
"ANTBTC.parquet",
"ATMBTC.parquet",
"AUCBTC.parquet",
"AVTBTC.parquet",
"BATBTC.parquet",
"BCHBTC.parquet",
"BCIBTC.parquet",
"BFTBTC.parquet",
"BNTBTC.parquet",
"BTGBTC.parquet",
"CBTBTC.parquet",
"CFIBTC.parquet",
"CNDBTC.parquet",
"CTXBTC.parquet",
"DADBTC.parquet",
"DAIBTC.parquet",
"DATBTC.parquet",
"DSHBTC.parquet",
"DTABTC.parquet",
"DTHBTC.parquet",
"EDOBTC.parquet",
"ELFBTC.parquet",
"EOSBTC.parquet",
"ESSBTC.parquet",
"ETCBTC.parquet",
"ETHBTC.parquet",
"ETPBTC.parquet",
"FSNBTC.parquet",
"FUNBTC.parquet",
"GNTBTC.parquet",
"HOTBTC.parquet",
"IOSBTC.parquet",
"IOTBTC.parquet",
"IQXBTC.parquet",
"KNCBTC.parquet",
"LRCBTC.parquet",
"LTCBTC.parquet",
"LYMBTC.parquet",
"MITBTC.parquet",
"MKRBTC.parquet",
"MNABTC.parquet",
"MTNBTC.parquet",
"NCABTC.parquet",
"NEOBTC.parquet",
"ODEBTC.parquet",
"OMGBTC.parquet",
"ORSBTC.parquet",
"PAIBTC.parquet",
"POABTC.parquet",
"POYBTC.parquet",
"QSHBTC.parquet",
"QTMBTC.parquet",
"RCNBTC.parquet",
"RDNBTC.parquet",
"REPBTC.parquet",
"REQBTC.parquet",
"RLCBTC.parquet",
"RRTBTC.parquet",
"SANBTC.parquet",
"SEEBTC.parquet",
"SENBTC.parquet",
"SNGBTC.parquet",
"SNTBTC.parquet",
"SPKBTC.parquet",
"STJBTC.parquet",
"TNBBTC.parquet",
"TRXBTC.parquet",
"UTKBTC.parquet",
"VEEBTC.parquet",
"VENBTC.parquet",
"WAXBTC.parquet",
"WPRBTC.parquet",
"XLMBTC.parquet",
"XMRBTC.parquet",
"XRPBTC.parquet",
"XVGBTC.parquet",
"YYWBTC.parquet",
"ZCNBTC.parquet",
"ZECBTC.parquet",
"ZILBTC.parquet",
"ZRXBTC.parquet"
]

In [14]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None

In [15]:
def derive_coin(filename, cash):
    file = filename.replace(".parquet", "")
    coin = file.replace(cash, "")
    return coin

In [14]:
sql_path = '../database/BitfinexData.db'
conn = sqlite3.connect(sql_path)

In [17]:
cursor      = conn.cursor()
cursor.execute("DROP Table IF EXISTS Bitfinex_USD")

In [18]:
frames = [];
for file in usd_files:
    filepath = "./rawdata/bitfinex/" + file
    table = pq.read_table(filepath, columns=['timestamp_ms', 'open', 'close', 'high', 'low', 'volume']).to_pandas()
    table['coin'] = derive_coin(file, "USD")
    table=table.rename(columns = {'timestamp_ms':'date'})
    table['date']=table['date']/1000
    frames.append(table)

In [19]:
main_df = pd.concat(frames)

In [15]:
engine = create_engine('sqlite:///'+sql_path)

In [16]:
main_df.to_sql('Bitfinex_USD', con=engine)

NameError: name 'main_df' is not defined

# Check SQL

In [25]:
usd_df = pd.read_sql('SELECT * FROM Bitfinex_USD',engine)

In [17]:
btc_df = pd.read_sql('SELECT * FROM Bitfinex_BTC',engine)

In [28]:
poloniex_df = pd.read_sql('SELECT * FROM History',engine)

In [18]:
end = int((btc_df.iloc[btc_df['date'].idxmax()])['date'])

In [19]:
start = int((btc_df.iloc[btc_df['date'].idxmin()])['date'])

In [20]:
end

1534555500

# load

In [ ]:
sql = 'SELECT coin, SUM(volume) AS total_volume FROM {table} WHERE date>=? and date<=? GROUP BY coin ORDER BY total_volume DESC LIMIT ?;'.format(table="Bitfinex_BTC");

In [ ]:
cursor=conn.cursor()

In [ ]:
cursor.execute(sql,(int(start), int(end), 15))

In [ ]:
coins_tuples = cursor.fetchall()

In [ ]:
conn.commit()
conn.close()

In [ ]:
coins_tuples

In [5]:
import sys
sys.path.append('/home/thorad/Core/Projects/AxiomAgent')

# check

In [28]:
import axiom.marketdata.globaldatamatrix as gdm
import numpy as np
import pandas as pd
import logging
from axiom.tools.configprocess import parse_time
from axiom.tools.data import get_volume_forward, get_type_list
import axiom.marketdata.replaybuffer as rb

In [29]:
volume_forward = get_volume_forward(end-start, 0.15, False)
history_manager = gdm.HistoryManager(
            coin_number=15,
            end=end,
            volume_average_days=30,
            volume_forward=volume_forward,
            online=False,
            table='Bitfinex_BTC',
            db_file='BitfinexData.db'
        )


In [ ]:
from axiom.tools.data import get_type_list
type_list = get_type_list(3)
global_panel = history_manager.get_global_panel(
                start,
                end,
                period=300,
                features=type_list
        )

In [32]:
global_panel.to_frame()

close      high       low
major minor                                            
XRP   2013-05-23 12:05:00  0.000190  0.000190  0.000190
      2013-05-23 12:10:00  0.000190  0.000190  0.000190
      2013-05-23 12:15:00  0.000190  0.000190  0.000190
      2013-05-23 12:20:00  0.000190  0.000190  0.000190
      2013-05-23 12:25:00  0.000190  0.000190  0.000190
      2013-05-23 12:30:00  0.000190  0.000190  0.000190
      2013-05-23 12:35:00  0.000190  0.000190  0.000190
      2013-05-23 12:40:00  0.000190  0.000190  0.000190
      2013-05-23 12:45:00  0.000190  0.000190  0.000190
      2013-05-23 12:50:00  0.000190  0.000190  0.000190
      2013-05-23 12:55:00  0.000190  0.000190  0.000190
      2013-05-23 13:00:00  0.000190  0.000190  0.000190
      2013-05-23 13:05:00  0.000190  0.000190  0.000190
      2013-05-23 13:10:00  0.000190  0.000190  0.000190
      2013-05-23 13:15:00  0.000190  0.000190  0.000190
      2013-05-23 13:20:00  0.000190  0.000190  0.000190
      2013-05-23 13:25:00  0.000190  0.000190  0.000190
      2013-05-23 13:30:00  0.000190  0.000190  0.000190
      2013-05-23 13:35:00  0.000190  0.000190  0.000190
      2013-05-23 13:40:00  0.000190  0.000190  0.000190
      2013-05-23 13:45:00  0.000190  0.000190  0.000190
      2013-05-23 13:50:00  0.000190  0.000190  0.000190
      2013-05-23 13:55:00  0.000190  0.000190  0.000190
      2013-05-23 14:00:00  0.000190  0.000190  0.000190
      2013-05-23 14:05:00  0.000190  0.000190  0.000190
      2013-05-23 14:10:00  0.000190  0.000190  0.000190
      2013-05-23 14:15:00  0.000190  0.000190  0.000190
      2013-05-23 14:20:00  0.000190  0.000190  0.000190
      2013-05-23 14:25:00  0.000190  0.000190  0.000190
      2013-05-23 14:30:00  0.000190  0.000190  0.000190
...                             ...       ...       ...
RRT   2018-08-17 23:00:00  0.000006  0.000006  0.000006
      2018-08-17 23:05:00  0.000006  0.000006  0.000006
      2018-08-17 23:10:00  0.000006  0.000006  0.000006
      2018-08-17 23:15:00  0.000006  0.000006  0.000006
      2018-08-17 23:20:00  0.000006  0.000006  0.000006
      2018-08-17 23:25:00  0.000006  0.000006  0.000006
      2018-08-17 23:30:00  0.000006  0.000006  0.000006
      2018-08-17 23:35:00  0.000006  0.000006  0.000006
      2018-08-17 23:40:00  0.000006  0.000006  0.000006
      2018-08-17 23:45:00  0.000006  0.000006  0.000006
      2018-08-17 23:50:00  0.000006  0.000006  0.000006
      2018-08-17 23:55:00  0.000006  0.000006  0.000006
      2018-08-18 00:00:00  0.000006  0.000006  0.000006
      2018-08-18 00:05:00  0.000006  0.000006  0.000006
      2018-08-18 00:10:00  0.000006  0.000006  0.000006
      2018-08-18 00:15:00  0.000006  0.000006  0.000006
      2018-08-18 00:20:00  0.000006  0.000006  0.000006
      2018-08-18 00:25:00  0.000006  0.000006  0.000006
      2018-08-18 00:30:00  0.000006  0.000006  0.000006
      2018-08-18 00:35:00  0.000006  0.000006  0.000006
      2018-08-18 00:40:00  0.000006  0.000006  0.000006
      2018-08-18 00:45:00  0.000006  0.000006  0.000006
      2018-08-18 00:50:00  0.000006  0.000006  0.000006
      2018-08-18 00:55:00  0.000006  0.000006  0.000006
      2018-08-18 01:00:00  0.000006  0.000006  0.000006
      2018-08-18 01:05:00  0.000006  0.000006  0.000006
      2018-08-18 01:10:00  0.000006  0.000006  0.000006
      2018-08-18 01:15:00  0.000006  0.000006  0.000006
      2018-08-18 01:20:00  0.000006  0.000006  0.000006
      2018-08-18 01:25:00  0.000006  0.000006  0.000006

[8262255 rows x 3 columns]

# Training

In [ ]:
import pgportfolio.autotrain.training
pgportfolio.autotrain.training.train_all(1, "cpu")